## CSCI 470 Activities and Case Studies

1. For all activities, you are allowed to collaborate with a partner. 
1. For case studies, you should work individually and are **not** allowed to collaborate.

By filling out this notebook and submitting it, you acknowledge that you are aware of the above policies and are agreeing to comply with them.

Some considerations with regard to how these notebooks will be graded:

1. Cells in which "# YOUR CODE HERE" is found are the cells where your graded code should be written.
2. In order to test out or debug your code you may also create notebook cells or edit existing notebook cells other than "# YOUR CODE HERE". We actually highly recommend you do so to gain a better understanding of what is happening. However, during grading, **these changes are ignored**. 
3. You must ensure that all your code for the particular task is available in the cells that say "# YOUR CODE HERE"
4. Every cell that says "# YOUR CODE HERE" is followed by a "raise NotImplementedError". You need to remove that line. During grading, if an error occurs then you will lose points for your work in that section.
5. If your code passes the "assert" statements, then no output will result. If your code fails the "assert" statements, you will get an "AssertionError". Getting an assertion error means you will not receive points for that particular task.
6. If you edit the "assert" statements to make your code pass, they will still fail when they are graded since the autograder will ignore the modified "assert" statement. Make sure you don't edit the assert statements.
7. We may sometimes have "hidden" tests for grading. This means that passing the visible "assert" statements is not sufficient. The "assert" statements are there as a guide but you need to make sure you understand what you're required to do and ensure that you are doing it correctly. Passing the visible tests is necessary but not sufficient to get the grade for that cell.
8. When you are asked to define a function, make sure you **don't** use any variables outside of the parameters passed to the function. You can think of the parameters being passed to the function as a hint. Make sure you're using all of those variables.
9. The **Grading** section at the end of the document (before the **Feedback** section) contains some code for our autograder on GradeScope. You are expected to fail this block of code in your Jupyter environment. DO NOT edit this block of code, or you may not get points for your assignment.
10. Finally, **make sure you run "Kernel > Restart and Run All"** and pass all the asserts before submitting. If you don't restart the kernel, there may be some code that you ran and deleted that is still being used and that was why your asserts were passing.

# Feature Learning

In this exercise we will run feature learning methods on a dataset to identify the key features to use for predicting a target within it.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn as sk

plt.style.use("ggplot")

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, mutual_info_regression, RFE
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

The data we'll be working with is the [California housing dataset](https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset).

In [ ]:
house_data = fetch_california_housing()

In [ ]:
print(house_data["DESCR"])

By reading the above information about the data, which features do you intuit meaningful for predicting the target? Which are not meaningful?

In [ ]:
house_features = pd.DataFrame(house_data["data"], columns=house_data["feature_names"])
house_prices = pd.Series(house_data["target"])

In [ ]:
house_features.head()

In [ ]:
house_features.describe()

In [ ]:
house_prices.head()

In [ ]:
house_prices.describe()

First, we'll [standardize](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling) our data to be used by our model. We'll use the [StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler) which just adjusts the mean to be 0 and scales the variance of the data.

This class is a scikit-learn transformer which follows a two phase process:
1. First it fits to the data to learn something (in this case how to scale the data)
1. Then it transforms any desired dataset based on the scale it learned.

We also have a helper function that is `fit_transform` that allows us to do both processes at the same time.

Now since we only ever want to fit our model once, we should never use fit_transform more than once. We can use `fit_transform` on training data but then just `transform` the test data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(house_features, house_prices, test_size=0.2, random_state=0)

In [ ]:
# Use a StandardScaler to standardize the house_features.
# Update (overwrite) the older values of X_train and X_test with the
# scaled values that are output by the StandardScaler object.
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
X_test.mean(axis = 0)

In [ ]:
assert np.all(X_train.mean(axis=0)< 0.0000000001)

# Note that if you do a fit_transform on the test data you did that part wrong
assert np.all(np.abs(X_test.mean(axis=0)) > 0.0000001)
assert np.all(np.abs(X_test.mean(axis=0)) < 0.04) 

Next, we'll split our data in order to follow the process we had outlined several lectures ago for effectively evaluating supervised learning problems.

## Filter Feature Selection

Select the best features based on [mutual information score](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html#sklearn.feature_selection.mutual_info_regression) from the __training data__, then transform X_train and X_test into the new subset of selected features. See [SelectKBest](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest).

In [ ]:
# We will select the best k features from all feature selection methods
k = 4

In [ ]:
# Save the transformer you use (fit) as "mi_transformer".
# Save the transformed set of features as as "mi_X_train" and "mi_X_test".
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
for feature, importance in zip(house_features.columns, mi_transformer.scores_):
    print(f"The MI score for {feature} is {importance}")

Do these values match what you expected? What are the most important features to use for predicting house prices?

In [ ]:
assert mi_transformer.k == k
assert isinstance(mi_transformer, SelectKBest)
assert len(mi_transformer.scores_) == 8
assert mi_X_train.shape == (16512, k)
assert mi_X_test.shape == (4128, k)

Since the focus in this exercise is on the feature learning and not on the supervised learning portion, we will use a simple estimator (linear regression) for the model training portions.

In [ ]:
miEst = LinearRegression().fit(mi_X_train, y_train)

In [ ]:
print(f"The mean squared error when training on the MI selected features is {mean_squared_error(y_train, miEst.predict(mi_X_train))}.")
print(f"When testing on the test data, the mean squared error is {mean_squared_error(y_test, miEst.predict(mi_X_test))}")

## Wrapper Feature Selection

Now try using [recursive feature elimination](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html) to select the 4 features we will use instead.

Note that after calling the ```.fit()``` method of the RFE object, a Boolean array that indicates which features are selected is stored as ```rfe_transformer.support_```.

In [ ]:
# Use an RFE object to determine the k features to select from X_train using a step of 2
# Save the rfe object as rfe_transformer
# Create rfe_X_train and rfe_X_test as the updated features based on the RFE output

rfeEst = LinearRegression()

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert isinstance(rfe_transformer, RFE)
assert rfe_transformer.step == 2
assert len(rfe_transformer.support_) == 8
assert rfe_X_train.shape == (16512, k)
assert rfe_X_test.shape == (4128, k)

In [ ]:
rfeEst = LinearRegression().fit(rfe_X_train, y_train)

In [ ]:
print(f"The mean squared error when training on the RFE selected features is {mean_squared_error(y_train, rfeEst.predict(rfe_X_train))}.")
print(f"When testing on the test data, the mean squared error is {mean_squared_error(y_test, rfeEst.predict(rfe_X_test))}")

In [ ]:
print(f"The most important features as determined by RFE were {list(house_features.columns[rfe_transformer.support_])}")

## Embedded Methods

For the embedded methods feature selection example, we will use Lasso. For this task you should use [LassoCV](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html#sklearn.linear_model.LassoCV) and **not** [Lasso](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) so that it trains with various values for alpha.

Since this is an embedded method, the feature selection will occur directly in the model.

In [ ]:
# Create a LassoCV model trained with 10 alphas and save it to lassoEst
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
lassoEst.coef_

In [ ]:
for feature, coef in zip(house_features.columns, lassoEst.coef_):
    print(f"The magniture of the feature coefficient for {feature} is {abs(coef)}.")

In [ ]:
lassoEst.alpha_

In [ ]:
assert lassoEst
assert isinstance(lassoEst, LassoCV)
assert len(lassoEst.coef_) == 8

In [ ]:
print(f"The mean squared error when training using lasso is {mean_squared_error(y_train, lassoEst.predict(X_train))}.")
print(f"When testing on the test data, the mean squared error is {mean_squared_error(y_test, lassoEst.predict(X_test))}")

Compare each model's prioritized features. Which model do you think is the best? What do they tell you about this data?

# Grading
The following code block is purely used for grading. If you find any error, you can ignore. DO NOT MODIFY THE CODE BLOCK BELOW.

In [ ]:
# Autograding with Otter Grader
import otter
grader = otter.Notebook()
grader.check_all()

## Feedback

In [ ]:
def feedback():
    """Provide feedback on the contents of this exercise
    
    Returns:
        string
    """
    # YOUR CODE HERE
    raise NotImplementedError()